In [1]:
from audio_encoder import AudioEncoder
from audio_diffusion import AudioDiffusion
from audio_pipline import AudioDiffusionPipeline
import torch
from IPython.display import Audio

device = "cuda" if torch.cuda.is_available() else "cpu"
generator = torch.Generator(device=device)

audio_encoder = AudioEncoder.from_pretrained("teticio/audio-encoder")

/opt/anaconda3/envs/metal/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import urllib
import requests
import os

# Get temporary API credentials
credentials = requests.get(
    "https://open.spotify.com/get_access_token?reason=transport&productType=embed"
).json()
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer " + credentials["accessToken"]
}

# Search for tracks
search_string = input("Search: ")
response = requests.get(
    f"https://api.spotify.com/v1/search?q={urllib.parse.quote(search_string)}&type=track",
    headers=headers).json()

# List results
for _, track in enumerate(response["tracks"]["items"]):
    print(f"{_ + 1}. {track['artists'][0]['name']} - {track['name']}")
selection = input("Select a track: ")

# Download and encode selection
r = requests.get(response["tracks"]["items"][int(selection) -
                                             1]["preview_url"],
                 stream=True)
with open("temp.mp3", "wb") as f:
    for chunk in r:
        f.write(chunk)
encoding = torch.unsqueeze(audio_encoder.encode(["temp.mp3"]),
                           axis=1).to(device)

os.remove("temp.mp3")

Search:  chopin


1. Frédéric Chopin - Nocturne Op. 9 No. 2
2. Frédéric Chopin - Chopin: Nocturne No. 20 in C-Sharp Minor, Op. Posth.
3. Frédéric Chopin - Ballade No. 1 in G Minor, Op. 23
4. Frédéric Chopin - Chopin: 24 Preludes, Op. 28: No. 4 in E Minor
5. Frédéric Chopin - Prélude in E minor, Op. 28, No. 4
6. Frédéric Chopin - Chopin: 24 Preludes, Op. 28: No. 4 in E Minor
7. Frédéric Chopin - Barcarolle in F-Sharp Major, Op. 60
8. Frédéric Chopin - Chopin: 24 Preludes, Op. 28: No. 9 in E Major
9. Frédéric Chopin - Chopin: Nocturne No. 13 in C Minor, Op. 48 No. 1
10. Frédéric Chopin - Fantasie-impromptu in C-Sharp Minor, Op. 66 No. 4
11. Frédéric Chopin - Chopin: Waltz No. 13 in D-Flat Major, Op. Posth. 70 No. 3
12. Frédéric Chopin - Chopin: Nocturne No. 2 in E-Flat Major, Op. 9 No. 2
13. Frédéric Chopin - Etude Op. 25 : No. 11 in A Minor
14. Frédéric Chopin - Chopin: 24 Preludes, Op. 28: No. 2 in A Minor
15. Frédéric Chopin - Chopin: Nocturne No. 2 in E-Flat Major, Op. 9 No. 2
16. Frédéric Chopin - Wa

Select a track:  1


In [ ]:
audio_diffusion = AudioDiffusion(model_id="Nkumar5/CAGHiphop", trust_remote_code=True)

generator.manual_seed(42)

image, (sample_rate, audio) = audio_diffusion.generate_spectrogram_and_audio(
    generator=generator, encoding=encoding)

display(image)
display(Audio(audio, rate=sample_rate))

Keyword arguments {'trust_remote_code': True} are not expected by AudioDiffusionPipeline and will be ignored.
  0%|          | 0/1000 [00:00<?, ?it/s]